In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from tqdm import tqdm 
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import logging
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoModel,
    AutoConfig,
)
print("total GPU memory:", torch.cuda.mem_get_info()[1] / 1024 ** 3)
print("available GPU memory:", torch.cuda.mem_get_info()[0] / 1024 ** 3)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    logging.warning("No GPU available, using CPU will be super slow")

total GPU memory: 7.7939453125
available GPU memory: 7.1591796875


### Load dataset, model, and tokenizer

In [2]:
dataset_id = "ag_news"
model_id = "roberta-base"

In [3]:
dataset = load_dataset(dataset_id)

In [4]:
train_dataset = dataset['train']
test_dataset = dataset["test"].shard(num_shards=2, index=0)
val_dataset = dataset["test"].shard(num_shards=2, index=1)

In [5]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256, return_tensors="pt")

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/3800 [00:00<?, ? examples/s]

In [6]:
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names

print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


In [7]:
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [10]:
roberta_model = RobertaForSequenceClassification.from_pretrained(model_id, config=config).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluate performance before fine-tuning

In [11]:
roberta_predictions = []
for i in tqdm(range(len(test_dataset["text"])), total=len(test_dataset["text"])):
    test_input = tokenizer(test_dataset["text"][i], return_tensors="pt").to(device)
    with torch.no_grad():
        logits = roberta_model(**test_input).logits
    predicted_class_id = logits.argmax().item()
    roberta_predictions.append(predicted_class_id)

100%|██████████| 3800/3800 [00:29<00:00, 130.02it/s]


In [12]:
print("raw roberta accuracy: ", round(accuracy_score(test_dataset["label"], roberta_predictions), 3))

raw roberta accuracy:  0.251


### Fine-tune Roberta

In [13]:
training_args = TrainingArguments(
    output_dir="./result/fine_tuned_agnews",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=180,
    per_device_eval_batch_size=380,
    learning_rate=0.005,
    weight_decay=0.01,
    logging_strategy="steps",
    warmup_steps=50,
    eval_steps=100,
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [14]:
trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [15]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB (GPU 0; 7.79 GiB total capacity; 6.66 GiB already allocated; 39.00 MiB free; 6.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.evaluate()

{'eval_loss': 1.3865007162094116,
 'eval_runtime': 12.3636,
 'eval_samples_per_second': 307.355,
 'eval_steps_per_second': 0.809,
 'epoch': 1.0}

In [ ]:
finetuned_roberta_outputs = trainer.predict(test_dataset)

In [ ]:
finetuned_roberta_predictions = finetuned_roberta_outputs[1]

In [ ]:
print("fine-tuned roberta accuracy: ", round(accuracy_score(test_dataset["label"], finetuned_roberta_predictions), 3))

fine-tuned roberta accuracy:  1.0


### Load fine-tuned Roberta

In [ ]:
finetuned_model = RobertaForSequenceClassification.from_pretrained("./result/fine_tuned_agnews/checkpoint-667/", local_files_only=True)